In [1]:
import pandas as pd
import numpy as np

from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from bs4 import BeautifulSoup
from requests_html import HTMLSession

import time

In [2]:
driver = webdriver.Chrome(executable_path='C:/Users/17178/Downloads/chromedriver_win32/chromedriver.exe')
driver.get('https://www.autonation.com/cars-for-sale?pagesize=72&cnd=used&bd=sedan')
content = driver.page_source
soup = BeautifulSoup(content)

In [3]:
time.sleep(5)

#have to re-load for whatever reason, else manufacturer dropdown doesn't expand
driver.get('https://www.autonation.com/cars-for-sale?pagesize=72&cnd=used&bd=sedan')


#accept cookies (uncovers load more bar)
cookies = driver.find_element_by_xpath('/html/body/div[4]/div[3]/button')
cookies.click()

time.sleep(5)

#read in data by manufacturer
for i in range(1,38):
    driver.get('https://www.autonation.com/cars-for-sale?pagesize=72&cnd=used&bd=sedan')

    #show manufacturer options
    show_manufacturers = driver.find_element_by_xpath("/html/body/section/div[4]/an-root/ansrp-dtansearchresultspagelib/srp-root/div/div/an-search/div[1]/div[3]/div[1]/div[3]/an-srp-refiners/div[1]/an-panel[3]/div")
    show_manufacturers.click()

    time.sleep(5)
        
    manufacturer_filter = driver.find_element_by_xpath(f"/html/body/section/div[4]/an-root/ansrp-dtansearchresultspagelib/srp-root/div/div/an-search/div[1]/div[3]/div[1]/div[3]/an-srp-refiners/div[1]/an-panel[3]/div/div[2]/div/an-body/an-srp-refiner-make/div/div[{i}]")
    manufacturer_filter.click()

    time.sleep(5)

    while True:
        try:
            showmore = driver.find_element_by_xpath("/html/body/section/div[4]/an-root/ansrp-dtansearchresultspagelib/srp-root/div/div/an-search/div[1]/div[3]/div[3]/div[2]/div[1]/button")
            showmore.click()
            time.sleep(5)
        except NoSuchElementException:
            break  

    content = driver.page_source
    soup = BeautifulSoup(content)

    results_model_trim = []
    results_price = []


    counter = 0

    for element in soup.findAll('div', {'class': 'col-12 px-0 resultsSRP'}):
        for model_trim in element.findAll('div', {'class': 'vehicle-stylename'}):
            results_model_trim.append(model_trim.contents[0])
            counter += 1
        for price in element.findAll('div', {'class': 'price-Value ml-auto ng-star-inserted'}):
            results_price.append(price.contents[0])


    results_year_make = []
    results_state = []
    results_miles  = []

    for x in range(0,counter):
        for year_make in element.findAll('a', {'id': f'srp-tile-vehiclename-{x}'}):
            results_year_make.append(year_make.contents[0])
        for state in soup.findAll('span', {'id': f'srp-tile-distance-{x}'}):
            results_state.append(state.contents[0][1:3])
        if element.findAll('span', {'id': f'srp-tile-mileage-{x}'}):
            miles = element.find('span', {'id': f'srp-tile-mileage-{x}'})
            results_miles.append(miles.contents[0])
        else:
            results_miles.append('missing')

    df_temp = pd.DataFrame(
            {'year_make': results_year_make,
             'model_trim': results_model_trim,
             'price': results_price,
             'state': results_state,
             'miles': results_miles,
             'source': 'autonation',
             'type': 'sedan'
            })

    if i == 1:
        df = df_temp
    else:
        df = pd.merge(df, df_temp, how='outer')
        del df_temp

df

,year_make,model_trim,price,state,miles,source,type
0,2016 Acura TLX,4dr Sdn FWD,"$18,495",MD,69K miles,autonation,sedan
1,2017 Acura TLX,FWD V6 w/Technology Pkg,"$22,995",VA,56K miles,autonation,sedan
2,2009 Acura TL,4dr Sdn SH-AWD,"$9,985",OH,87K miles,autonation,sedan
3,2009 Acura TSX,4dr Sdn Auto,"$7,989",OH,155K miles,autonation,sedan
4,2021 Acura TLX,FWD w/A-Spec Package,"$39,470",IL,3K miles,autonation,sedan
...,...,...,...,...,...,...,...
6055,2015 Volvo S60,4dr Sdn T5 Drive-E Premier FWD,"$15,995",CA,54K miles,autonation,sedan
6056,2001 Volvo S80,2.9 A SR 4dr Sdn w/Sunroof,"$4,888",CA,117K miles,autonation,sedan
6057,2009 Volvo S60,4dr Sdn 2.5T FWD,"$5,888",CA,192K miles,autonation,sedan
6058,2017 Volvo V60,T5 AWD Premier,"$23,995",CA,30K miles,autonation,sedan


In [4]:
#convert miles to numeric value
df['miles'] = df['miles'].str.extract('([0-9]+)').astype(float)*1000

#convert price to numeric value
df['price'] = df['price'].replace({'\$':''}, regex=True)
df['price'] = df['price'].replace({'\*':''}, regex=True)
df['price'] = df['price'].replace({'\,':''}, regex=True)
df['price'] = df['price'].astype(int)

df

,year_make,model_trim,price,state,miles,source,type
0,2016 Acura TLX,4dr Sdn FWD,18495,MD,69000.0,autonation,sedan
1,2017 Acura TLX,FWD V6 w/Technology Pkg,22995,VA,56000.0,autonation,sedan
2,2009 Acura TL,4dr Sdn SH-AWD,9985,OH,87000.0,autonation,sedan
3,2009 Acura TSX,4dr Sdn Auto,7989,OH,155000.0,autonation,sedan
4,2021 Acura TLX,FWD w/A-Spec Package,39470,IL,3000.0,autonation,sedan
...,...,...,...,...,...,...,...
6055,2015 Volvo S60,4dr Sdn T5 Drive-E Premier FWD,15995,CA,54000.0,autonation,sedan
6056,2001 Volvo S80,2.9 A SR 4dr Sdn w/Sunroof,4888,CA,117000.0,autonation,sedan
6057,2009 Volvo S60,4dr Sdn 2.5T FWD,5888,CA,192000.0,autonation,sedan
6058,2017 Volvo V60,T5 AWD Premier,23995,CA,30000.0,autonation,sedan


In [5]:
df.to_csv('C:/Users/17178/Documents/DS Projects/Nitorum Case/Sandbox/autonation.csv')